In [1]:
import os
import gc
import cv2
import torch
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from ultralytics.models import YOLO
from ultralytics.models.yolo.detect.train import DetectionTrainer
from ultralytics.cfg import cfg2dict, get_cfg
from ultralytics.data.build import build_dataloader, build_yolo_dataset

from utils.tools import (boxes_to_coor, compute_normalized_cross_correlation,
                         delete_min_object_iter, delete_temp_img,
                         save_loss, save_visualize_image)

# Utils

In [2]:
from ultralytics.utils.instance import  Instances 

def cut_images(shape, new_shape = [640,640]):
    x,y,w,h = 0.5,0.5,1,1
    gt = Instances(np.array([x, y, w, h]),bbox_format= 'xywh')
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    #r = min(r, 1.0)
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding
    dw /= 2  # divide padding into 2 sides
    dh /= 2
    ratio = r, r  # width, height ratios
    gt.convert_bbox(format='xyxy')
    gt.denormalize(*shape[:2][::-1])
    gt.scale(*ratio)
    gt.add_padding(dw, dh)
    bbox_after = gt._bboxes.bboxes.reshape(-1).astype(int)
    return bbox_after

# Code

In [3]:
yaml_path = "/home/son/Desktop/Research/ultralytics/ultralytics/cfg/my_config.yaml"
data_info_path = "/home/son/Desktop/Research/ultralytics/ultralytics/cfg/datasets/custom_VOC.yaml"
img_path = "/home/son/Desktop/Research/datasets/VOC/images/val2012/"

In [4]:
data_info = cfg2dict(data_info_path)
cfg = get_cfg(yaml_path)
voc_dataset = build_yolo_dataset(cfg=cfg, batch=1, img_path=img_path,data=data_info)
loader = build_dataloader(cfg=cfg, dataset=voc_dataset, batch=1, workers=cfg.workers)

trainer = DetectionTrainer(yaml_path)
trainer.setup_model()
trainer.set_model_attributes()
# trainer.model.train()
trainer.model.cuda()
trainer.model.train()

train: Scanning /home/son/Desktop/Research/datasets/VOC/labels/val2012.cache... 5823 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5823/5823 [00:00<?, ?it/s]


Ultralytics YOLOv8.0.207 🚀 Python-3.10.13 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 3902MiB)
engine/trainer: task=detect, mode=predict, model=yolov8n.pt, data=ultralytics/cfg/datasets/custom_VOC.yaml, epochs=1, patience=10, batch=1, imgsz=640, save=True, save_period=100, cache=False, device=0, workers=8, project=None, name=predict9, augment=False, exist_ok=False, pretrained=False, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=True, fraction=1.0, cos_lr=False, close_mosaic=10, resume=False, min_memory=False, overlap_mask=True, mask_ratio=4, dropout=0.0, YOLO_VERBOSE=False, val=True, split=val, save_json=False, save_hybrid=False, conf=0.25, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=True, save_conf=False, save_crop=False, hide_labels=True, hide_conf=True, vid_stride=1, line_thickness=3, visualize=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=Tr

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
    

In [7]:
folder_name = "results"
conf = 0.75
ncc_value = 0.6
iter_start = 0
iter_end = 2500


for i, batch in enumerate(loader[0]):
    if (i < iter_start):
        continue
    if (i > iter_end):
        break

    loss_log_image  = []
    path_img        = batch['im_file'][0]
    name_img        = os.path.split(path_img)[-1]

    print(f"[{i}/{len(loader[0])}] Attacking image {name_img} ...")

    batch_tensor    = torch.Tensor(batch['img'].float()).cuda()
    origin_tensor   = torch.Tensor(batch['img'].float()).cuda()
    img_before      = cv2.imread(path_img)
    x,y,_           = img_before.shape              # original img shape
    x_new,y_new     = batch_tensor.shape[-2:]       # shape img after augment with yolov8

    x1,y1,x2,y2     = cut_images((x,y,3),[x_new,y_new])
    
    batch_tensor.requires_grad_()
    batch_tensor.retain_grad()
    b = trainer.model(batch_tensor)
    loss, _ = trainer.model.loss(batch=batch, preds=b)

    loss.retain_grad()
    loss.backward()

    alpha = batch_tensor.grad

    model = YOLO(cfg.get('model'))
    model.conf = conf

    image_to_attack_ori = batch_tensor[:,:,y1:y2,x1:x2].detach().clone().cuda()
    image_to_attack = batch_tensor[:,:,y1:y2,x1:x2].cuda()
    attack_grad     = alpha[:,:,y1:y2,x1:x2].cuda()

    step = 100
    loss_log_image.append(loss)
    folder_result = folder_name +"_"+cfg.get('model')+"_"+ str(ncc_value) +'_ncc_' + str(conf)+"_" + str(iter_start) + "-"+ str(iter_end)
    num_object_min = 0
    num_iter_min   = 0

    for number_of_steps in range(60000):
        res = model.predict(path_img)
        number_object = res[0].boxes.xyxy.shape[0]
        print(number_object)
        print(res[0].boxes.conf)
                
        if number_of_steps == 0:
            save_visualize_image(image_to_attack/255, folder_result ,name_img, 'base')
            path_img           =   f'./img_results/{folder_result}/images_full/val/{name_img[:-4]}_base.jpg'
            step = 0
            num_object_min += number_object
            num_iter_min   += number_of_steps
            save_visualize_image(image_to_attack/255, folder_result ,name_img, f'{number_object}_{number_of_steps}')
            continue
            
        else:
            step = 100
        
        if number_object < num_object_min:
            num_object_min = 0
            num_iter_min   = 0
            num_object_min += number_object
            num_iter_min   += number_of_steps

        
        count = 0
        masks_object_attack                                            = torch.zeros_like(attack_grad)
        for index in range(number_object):
            x1_bbox,y1_bbox,x2_bbox,y2_bbox                            = boxes_to_coor(res[0].boxes.xyxy[index])
            ########################################
            image1 = image_to_attack[:,:,y1_bbox:y2_bbox,x1_bbox:x2_bbox] 
            image2 = image_to_attack_ori[:,:,y1_bbox:y2_bbox,x1_bbox:x2_bbox]
            ncc    = compute_normalized_cross_correlation(image1, image2)
            print(ncc)
            if ncc < ncc_value:
                count += 1
                continue        
            ########################################
            masks_object_attack[:,:,y1_bbox:y2_bbox,x1_bbox:x2_bbox]  += 1
            
        
        masks_object_attack                    = torch.where(masks_object_attack > 0, 1, 0)
        new_image                              = image_to_attack + step*attack_grad*masks_object_attack
        image_to_attack                        = new_image  
        
            
                
        save_visualize_image(new_image/255, folder_result ,name_img, f'{number_object}_{number_of_steps}')
        if count == number_object:
            break
        path_img           =   f'./img_results/{folder_result}/images_full/val/{name_img[:-4]}_{number_object}_{number_of_steps}.jpg'
        
        with torch.no_grad():
            batch_tensor[:,:,y1:y2,x1:x2] = image_to_attack
            
        batch_tensor.requires_grad_()
        batch_tensor.retain_grad()
        b = trainer.model(batch_tensor)

        loss, _ = trainer.model.loss(batch=batch, preds=b)
        

        loss.retain_grad()
        loss.backward()
        alpha = batch_tensor.grad
        image_to_attack = batch_tensor[:,:,y1:y2,x1:x2].cuda()
        attack_grad     = alpha[:,:,y1:y2,x1:x2].cuda()
    
        loss_log_image.append(loss)
        
        torch.cuda.empty_cache()
        gc.collect()
    delete_min_object_iter(folder_result, name_img, num_object_min, num_iter_min)
    save_loss(loss_log_image, folder_result, name_img)

[0/5823] Attacking image 2009_002335.jpg ...



image 1/1 /home/son/Desktop/Research/datasets/VOC/images/val2012/2009_002335.jpg: 640x448 1 motorcycle, 28.8ms
Speed: 1.2ms preprocess, 28.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 448)

image 1/1 /home/son/Desktop/Research/ultralytics/img_results/results_yolov8n.pt_0.6_ncc_0.75_0-2500/images_full/val/2009_002335_base.jpg: 640x448 (no detections), 4.7ms
Speed: 0.9ms preprocess, 4.7ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 448)


1
tensor([0.8373], device='cuda:0')
0
tensor([], device='cuda:0')



image 1/1 /home/son/Desktop/Research/datasets/VOC/images/val2012/2009_000412.jpg: 480x640 3 horses, 5.8ms
Speed: 1.5ms preprocess, 5.8ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/son/Desktop/Research/ultralytics/img_results/results_yolov8n.pt_0.6_ncc_0.75_0-2500/images_full/val/2009_000412_base.jpg: 480x640 1 horse, 5.4ms
Speed: 1.6ms preprocess, 5.4ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)


[1/5823] Attacking image 2009_000412.jpg ...
3
tensor([0.9163, 0.9023, 0.8969], device='cuda:0')
1
tensor([0.8603], device='cuda:0')
0.9999989093848167


...


image 1/1 /home/son/Desktop/Research/ultralytics/img_results/results_yolov8n.pt_0.6_ncc_0.75_0-2500/images_full/val/2008_002410_1_56.jpg: 640x576 1 cat, 5.5ms
Speed: 1.4ms preprocess, 5.5ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 576)

image 1/1 /home/son/Desktop/Research/ultralytics/img_results/results_yolov8n.pt_0.6_ncc_0.75_0-2500/images_full/val/2008_002410_1_57.jpg: 640x576 1 cat, 5.5ms
Speed: 1.1ms preprocess, 5.5ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 576)


1
tensor([0.8723], device='cuda:0')
0.9890281801449436
1
tensor([0.8764], device='cuda:0')
0.9885634228275378



image 1/1 /home/son/Desktop/Research/ultralytics/img_results/results_yolov8n.pt_0.6_ncc_0.75_0-2500/images_full/val/2008_002410_1_58.jpg: 640x576 1 cat, 5.5ms
Speed: 1.0ms preprocess, 5.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 576)

image 1/1 /home/son/Desktop/Research/ultralytics/img_results/results_yolov8n.pt_0.6_ncc_0.75_0-2500/images_full/val/2008_002410_1_59.jpg: 640x576 1 cat, 5.6ms
Speed: 1.1ms preprocess, 5.6ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 576)


1
tensor([0.8912], device='cuda:0')
0.9880888714565975
1
tensor([0.8576], device='cuda:0')
0.9876047768442986


KeyboardInterrupt: 

# Convert PASCAL VOC label to YOLO

In [2]:
import xml.etree.ElementTree as ET

from tqdm import tqdm
from pathlib import Path
import shutil

names_dict = {
  0: "aeroplane",
  1: "bicycle",
  2: "bird",
  3: "boat",
  4: "bottle",
  5: "bus",
  6: "car",
  7: "cat",
  8: "chair",
  9: "cow",
  10: "diningtable",
  11: "dog",
  12: "horse",
  13: "motorbike",
  14: "person",
  15: "pottedplant",
  16: "sheep",
  17: "sofa",
  18: "train",
  19: "tvmonitor",
}

def convert_label(path, lb_path, year, image_id):
    def convert_box(size, box):
        dw, dh = 1. / size[0], 1. / size[1]
        x, y, w, h = (box[0] + box[1]) / 2.0 - 1, (box[2] + box[3]) / 2.0 - 1, box[1] - box[0], box[3] - box[2]
        return x * dw, y * dh, w * dw, h * dh

    in_file = open(path / f'VOC{year}/Annotations/{image_id}.xml')
    out_file = open(lb_path, 'w')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    names = list(names_dict.values())  # names list
    for obj in root.iter('object'):
        cls = obj.find('name').text
        if cls in names and int(obj.find('difficult').text) != 1:
            xmlbox = obj.find('bndbox')
            bb = convert_box((w, h), [float(xmlbox.find(x).text) for x in ('xmin', 'xmax', 'ymin', 'ymax')])
            cls_id = names.index(cls)  # class id
            out_file.write(" ".join(str(a) for a in (cls_id, *bb)) + '\n')


# # Download
# dir = Path(yaml['path'])  # dataset root dir
# url = 'https://github.com/ultralytics/yolov5/releases/download/v1.0/'
# urls = [f'{url}VOCtrainval_06-Nov-2007.zip',  # 446MB, 5012 images
#         f'{url}VOCtest_06-Nov-2007.zip',  # 438MB, 4953 images
#         f'{url}VOCtrainval_11-May-2012.zip']  # 1.95GB, 17126 images
# download(urls, dir=dir / 'images', curl=True, threads=3)

dir = Path("../datasets/VOC")

# Convert
path = dir / 'images/VOCdevkit'
for year, image_set in ('2012', 'train'), ('2012', 'val'):
    imgs_path = dir / 'images' / f'{image_set}{year}'
    lbs_path = dir / 'labels' / f'{image_set}{year}'
    imgs_path.mkdir(exist_ok=True, parents=True)
    lbs_path.mkdir(exist_ok=True, parents=True)

    with open(path / f'VOC{year}/ImageSets/Main/{image_set}.txt') as f:
        image_ids = f.read().strip().split()
    # print(len(image_ids))
    for id in tqdm(image_ids, desc=f'{image_set}{year}'):
        f = path / f'VOC{year}/JPEGImages/{id}.jpg'  # old img path
        lb_path = (lbs_path / f.name).with_suffix('.txt')  # new label path
        shutil.move(f, imgs_path / f.name)
        # f.rename(imgs_path / f.name)  # move image
        convert_label(path, lb_path, year, id)  # convert labels to YOLO format

val2012: 100%|██████████| 5823/5823 [00:00<00:00, 6979.88it/s]
